In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 600)
pd.set_option('display.max_colwidth', 1000)

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device = 0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
data = pd.read_csv('three_hundred_sample.csv',index_col=0)
data = data[['CUSTOMER_RESPONSE', 'PROVIDER_NAME']]
data = data.head(100)

In [ ]:
labels = ['waiting for doctors',
          'long queues in hospital',
          'Authorization code',
          'delaying in getting test results',
         'attitude of nurses',
          'hospital related',
          'attitude of doctors',
          'medication quality',
          'cleanliness of hosptials',
          'front desk',
          'call center agents',
          'response to phone calls',
          'phone calls related',
          'Unavailable drug for pick-up at pharmacy',
          'medication pickup related',
          'lack of vitamin c in customer plan benefits',
          'lack of medical benefits in customer plan',
          'hospitals near the customer city',
          'lack of Paediatric care in customer plan benefits',
          'customer confusion about the benefits',
          'lack of communication on changing providers on plan',
          'internet work issue',
          'app dysfunction',
          'clinic related',
          'reception related',
          'doctor related',
          'delay in delivering',
          'quality of building',
          'hospital related',
          'customer visiting the hospital',
          'medication or treatment delivery',
          'cleanliness',
          'place hygeine',
          'customer service',
          'surgeon related',
          'drug delivery',
          'medication delivery',
          'access related',
          'time related',
          'waiting time',
          'reception related',
          'delivery related',
          'customer service related',
          'medication, treatment, or drug related',
          'delay'
          ]

In [ ]:
if not labels:
    raise ValueError("labels is empty. You must include at least one label.")

detailed_scores = []

# Use a safer loop in case there are fewer than 3000 entries or some are empty
for i in range(len(data)):
    input_text = data['CUSTOMER_RESPONSE'].iloc[i]

    if not input_text.strip():
        print(f"Skipping empty or whitespace 'CUSTOMER_RESPONSE' at index {i}.")
        continue

    try:
        model_dict = classifier(input_text, labels, multi_label=True)
        result_dict = dict(zip(model_dict.get('labels'), model_dict.get('scores')))
        score_dict = {'CUSTOMER_RESPONSE': input_text}  # using input_text directly
        score_dict.update(result_dict)
        detailed_scores.append(score_dict)

        print("Input Text for the model: ", input_text)
        for label in labels:
            print(f"Confidence Score for {label}: ", result_dict.get(label, "N/A"))  # Default to "N/A" if label not found
        print('\n\n')
    except Exception as e:
        # Log the exception and skip this iteration
        print(f"An error occurred at index {i} with text: {input_text}. Error: {e}")
        continue

detailed_scores_df = pd.DataFrame(detailed_scores)

Input Text for the model:  The healthcare practitioners I saw made me gave the rating
Confidence Score for waiting for doctors:  0.07291506975889206
Confidence Score for long queues in hospital:  0.004227184224873781
Confidence Score for Authorization code:  0.05012053623795509
Confidence Score for delaying in getting test results:  0.11966729909181595
Confidence Score for attitude of nurses:  0.5971038937568665
Confidence Score for hospital related:  0.6176520586013794
Confidence Score for attitude of doctors:  0.9126157164573669
Confidence Score for medication quality:  0.43220269680023193
Confidence Score for cleanliness of hosptials:  0.25601431727409363
Confidence Score for front desk:  0.00040039923624135554
Confidence Score for call center agents:  2.1716408809879795e-05
Confidence Score for response to phone calls:  0.008988486602902412
Confidence Score for phone calls related:  0.0004614243225660175
Confidence Score for Unavailable drug for pick-up at pharmacy:  0.021915106102

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Input Text for the model:  Yes they very very accommodating and very respectful in the case of treatment they're serious about what they do there staff are calming always so i love visiting them always
Confidence Score for waiting for doctors:  0.2024674266576767
Confidence Score for long queues in hospital:  0.00010513005690881982
Confidence Score for Authorization code:  0.3982512354850769
Confidence Score for delaying in getting test results:  0.03262190520763397
Confidence Score for attitude of nurses:  0.8550868034362793
Confidence Score for hospital related:  0.8107179999351501
Confidence Score for attitude of doctors:  0.9535074830055237
Confidence Score for medication quality:  0.7234782576560974
Confidence Score for cleanliness of hosptials:  0.1304294466972351
Confidence Score for front desk:  0.5761279463768005
Confidence Score for call center agents:  0.1798359900712967
Confidence Score for response to phone calls:  0.02178012952208519
Confidence Score for phone calls relat

In [ ]:
PreClassified = data
PreClassified['Health_benefits_coverage'] = ((detailed_scores_df['lack of vitamin c in customer plan benefits']>0.7).astype(int)|
                                        (detailed_scores_df['lack of Paediatric care in customer plan benefits']>0.9).astype(int)|
                                        (detailed_scores_df['lack of medical benefits in customer plan']>0.9).astype(int)
                                        )

PreClassified['provider_quality'] = ((detailed_scores_df['attitude of nurses'] > 0.95).astype(int)|
                                               (detailed_scores_df['attitude of doctors']>0.8).astype(int)|
                                            (detailed_scores_df['cleanliness of hosptials'] > 0.95).astype(int)|
                                     (detailed_scores_df['medication quality'] > 0.95).astype(int)|
                                          (detailed_scores_df['front desk'] > 0.95).astype(int)|
                                     (detailed_scores_df['reception related'] > 0.95).astype(int)|
                                     (detailed_scores_df['surgeon related'] > 0.95).astype(int)|
                                     (detailed_scores_df['place hygeine'] > 0.95).astype(int)|
                                     (detailed_scores_df['quality of building'] > 0.95).astype(int)|
                                     (detailed_scores_df['cleanliness'] > 0.99).astype(int)|
                                     (detailed_scores_df['customer visiting the hospital'] > 0.95).astype(int)|
                                     (detailed_scores_df['clinic related'] > 0.95).astype(int))

PreClassified['RCC_quality'] = ((detailed_scores_df['call center agents']>0.9).astype(int)|
                                        (detailed_scores_df['response to phone calls']>0.95).astype(int)|
                                        (detailed_scores_df['phone calls related']>0.9).astype(int))

PreClassified['provider_wait_times'] = ((detailed_scores_df['waiting for doctors'] > 0.9).astype(int) |
                                                (detailed_scores_df['long queues in hospital'] > 0.9).astype(int)|
                                               (detailed_scores_df['Authorization code'] > 0.9).astype(int)|
                                               (detailed_scores_df['delaying in getting test results'] > 0.99).astype(int))

PreClassified['medication_related'] = ((detailed_scores_df['Unavailable drug for pick-up at pharmacy']>0.9).astype(int)|
                                        (detailed_scores_df['medication pickup related']>0.9).astype(int))

PreClassified['customer_education'] = ( (detailed_scores_df['customer confusion about the benefits']>0.95).astype(int)|
                                              (detailed_scores_df['lack of communication on changing providers on plan']>0.95).astype(int))

PreClassified['provider_network'] = ( (detailed_scores_df['hospitals near the customer city']>0.95).astype(int)|
                                        (detailed_scores_df['lack of Paediatric care in customer plan benefits']>0.9).astype(int)
                                             )

PreClassified['tech_issues'] = ( (detailed_scores_df['internet work issue']>0.95).astype(int)|
                                              (detailed_scores_df['app dysfunction']>0.95).astype(int))

In [ ]:
PreClassified['wait_time_related'] = (detailed_scores_df['waiting time'] > 0.99).astype(int)
PreClassified['time related'] = (detailed_scores_df['time related'] > 0.99).astype(int)
PreClassified['access related'] = (detailed_scores_df['access related'] > 0.99).astype(int)
PreClassified['doctor related'] = (detailed_scores_df['doctor related'] > 0.99).astype(int)
PreClassified['clinic related'] = (detailed_scores_df['clinic related'] > 0.99).astype(int)
PreClassified['delivery related'] = (detailed_scores_df['delivery related'] > 0.98).astype(int)
PreClassified['customer service related'] = (detailed_scores_df['customer service'] > 0.99).astype(int)
PreClassified['delay'] = (detailed_scores_df['delay'] > 0.99).astype(int)
PreClassified["medication, treatment, or drug related"] = (detailed_scores_df["medication, treatment, or drug related"] > 0.9).astype(int)

In [ ]:
PreClassified['medication_delivery'] = 0
# If the customer response got classified as ("medication, drug, treatment" or medication relaed) related and it's not related to delivery --> then it's related to medicatuion quality aka provider provider.
PreClassified.loc[(PreClassified['medication, treatment, or drug related'] == 1) & (PreClassified['delivery related'] == 0), 'provider_quality'] = 1
PreClassified.loc[(PreClassified['medication_related'] == 1) & (PreClassified['delivery related'] == 0), 'provider_quality'] = 1

# if the customer response contains the word "customer service", then it's RCC related
PreClassified.loc[PreClassified['CUSTOMER_RESPONSE'].str.contains("customer service", case=False, na=False), 'RCC_quality'] = 1

# If the customer response contains "mosquitoes" then it's related to equipment, which means it's related to provider quality.
PreClassified.loc[PreClassified['CUSTOMER_RESPONSE'].str.contains("mosquitoes", case=False, na=False), 'provider_quality'] = 1

# if the customer response contains "during my visit", then it means that the customer went there to hospital, clinic..etc. which means provider quality not RCC
PreClassified.loc[PreClassified['CUSTOMER_RESPONSE'].str.contains("during my visit", case=False, na=False), 'provider_quality'] = 1
PreClassified.loc[PreClassified['CUSTOMER_RESPONSE'].str.contains("during my visit", case=False, na=False), 'RCC_quality'] = 0

# If the customer response got classified as doctor related, then it's a provider_quality not RCC
PreClassified.loc[PreClassified['doctor related'] == 1, 'provider_quality'] = 1

# If the provider name has "Eye", "GYM", "SPA", "EYE", "Dental", or "Clinic" then it's going to be a provider_quality and not RCC thing, and it's better to break it down to each category.
PreClassified.loc[PreClassified['PROVIDER_NAME'].str.contains("Eye", case=False, na=False), 'provider_quality'] = 1
PreClassified.loc[PreClassified['PROVIDER_NAME'].str.contains("Eye", case=False, na=False), 'RCC_quality'] = 0
PreClassified.loc[PreClassified['PROVIDER_NAME'].str.contains("Clinic", case=False, na=False), 'provider_quality'] = 1
PreClassified.loc[PreClassified['PROVIDER_NAME'].str.contains("Clinic", case=False, na=False), 'RCC_quality'] = 0
PreClassified.loc[PreClassified['PROVIDER_NAME'].str.contains("Dental", case=False, na=False), 'provider_quality'] = 1
PreClassified.loc[PreClassified['PROVIDER_NAME'].str.contains("Dental", case=False, na=False), 'RCC_quality'] = 0

# If the provider_name has "Pharmace" and the customer response got classified as delivery related then it's a medication delivery.
PreClassified.loc[(PreClassified['PROVIDER_NAME'].str.contains("Pharmacy", case=False, na=False)) & (PreClassified['delivery related'] == 1), 'medication_delivery'] = 1

# If the customer response contains "gym" then provider_quality=1
PreClassified.loc[PreClassified['CUSTOMER_RESPONSE'].str.contains("gym", case=False, na=False), 'provider_quality'] = 1
PreClassified.loc[PreClassified['CUSTOMER_RESPONSE'].str.contains("gym", case=False, na=False), 'RCC_quality'] = 0

# If the customer_response got classified as "is access" and "time related" then it's gonna be provider_wait_time
PreClassified.loc[(PreClassified['access related'] == 1) & (PreClassified['time related'] == 1), 'provider_wait_times'] = 1

# if it's delay related and RCC_quality = 0, then it's a provider wait time.
PreClassified.loc[(PreClassified['delay'] == 1) & (PreClassified['RCC_quality'] == 0), 'provider_wait_times'] = 1

# If the provider_name is Reliance family, then it's related to RCC
PreClassified.loc[PreClassified['PROVIDER_NAME'].str.contains("Reliance", case=False, na=False), 'RCC_quality'] = 1
PreClassified.loc[PreClassified['PROVIDER_NAME'].str.contains("Reliance", case=False, na=False), 'provider_quality'] = 0


In [ ]:
readable_NPS_data = PreClassified[['CUSTOMER_RESPONSE', 'PROVIDER_NAME',
                                   'Health_benefits_coverage', 'provider_quality',
                                   'RCC_quality', 'provider_wait_times',
                                   'customer_education', 'provider_network', 'medication_delivery']]
columns_to_check = ['Health_benefits_coverage', 'provider_quality',
                                   'RCC_quality', 'provider_wait_times',
                                   'customer_education', 'provider_network', 'medication_delivery']
def get_label(row):
    labels = [col for col in columns_to_check if row[col] == 1]
    return labels if labels else ["general"]

readable_NPS_data['label'] = readable_NPS_data.apply(get_label, axis=1)

<ipython-input-15-d1e88f734531>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readable_NPS_data['label'] = readable_NPS_data.apply(get_label, axis=1)


In [ ]:
explainable_NPS = readable_NPS_data[['CUSTOMER_RESPONSE', 'PROVIDER_NAME' ,'label']]

In [ ]:
readable_NPS_data

In [ ]:
subtopics_labels = ['nurses', 'doctors', 'surgeon', 'front desk',
                    'place hygiene', 'quality of building', 'cleanliness', 'reception',
                    'Unavailable drug for pick-up at pharmacy', 'medication pickup related']

In [ ]:
service_type_df = pd.read_csv('/content/Percentaged_NPS_DATA (2).csv')

In [ ]:
service_type_df = service_type_df[['PROVIDER_NAME', 'TYPE_SERVICE']]

In [ ]:
NPS_TOPIC = pd.merge(readable_NPS_data, service_type_df, left_on = 'PROVIDER_NAME',
                          right_on = 'PROVIDER_NAME')

In [ ]:
NPS_TOPIC.drop('label', inplace=True, axis=1)

In [ ]:
NPS_TOPIC.drop_duplicates(inplace=True)

In [ ]:
NPS_TOPIC.shape

(96, 10)

In [ ]:
NPS_TOPIC['GYM_Service'] = 0
NPS_TOPIC['DIAGNOSITC_Service'] = 0
NPS_TOPIC['PHARMACY_Service'] = 0
NPS_TOPIC['OPTICAL_Service'] = 0
NPS_TOPIC['DENTAL_Service'] = 0
NPS_TOPIC['DERMATOLOGY_Service'] = 0
NPS_TOPIC['ORTHOPEDIC_Service'] = 0
NPS_TOPIC['HOSPITAL_Service'] = 0
NPS_TOPIC['CLINIC_Service'] = 0

NPS_TOPIC.loc[NPS_TOPIC['TYPE_SERVICE'].str.contains("Gym", case=False, na=False), 'GYM_Service'] = 1
NPS_TOPIC.loc[NPS_TOPIC['TYPE_SERVICE'].str.contains("Diagnostic Center", case=False, na=False), 'DIAGNOSITC_Service'] = 1
NPS_TOPIC.loc[NPS_TOPIC['TYPE_SERVICE'].str.contains("Pharmacy", case=False, na=False), 'PHARMACY_Service'] = 1
NPS_TOPIC.loc[NPS_TOPIC['TYPE_SERVICE'].str.contains("Optical Center", case=False, na=False), 'OPTICAL_Service'] = 1
NPS_TOPIC.loc[NPS_TOPIC['TYPE_SERVICE'].str.contains("Dental Clinic", case=False, na=False), 'DENTAL_Service'] = 1
NPS_TOPIC.loc[NPS_TOPIC['TYPE_SERVICE'].str.contains("Dermatological Center", case=False, na=False), 'DERMATOLOGY_Service'] = 1
NPS_TOPIC.loc[NPS_TOPIC['TYPE_SERVICE'].str.contains("Orthopedic Center", case=False, na=False), 'ORTHOPEDIC_Service'] = 1
NPS_TOPIC.loc[NPS_TOPIC['TYPE_SERVICE'].str.contains("Hospita", case=False, na=False), 'HOSPITAL_Service'] = 1
NPS_TOPIC.loc[NPS_TOPIC['TYPE_SERVICE'].str.contains("Clinic", case=False, na=False), 'CLINIC_Service'] = 1


In [ ]:
NPS_TOPIC.to_csv('NPS_TOPIC.csv')

In [ ]:
subtopics_labels = ['nurses', 'doctors', 'surgeon', 'front desk',
                    'place hygiene', 'quality of building', 'cleanliness', 'reception',
                    'Unavailable drug for pick-up at pharmacy', 'medication pickup related']

In [ ]:
if not subtopics_labels:
    raise ValueError("labels is empty. You must include at least one label.")

detailed_scores = []
#provider_quality_data = explainable_NPS[explainable_NPS['has_provider_quality']==True]
# Use a safer loop in case there are fewer than 3000 entries or some are empty
for i in range(len(NPS_TOPIC)):
    input_text = NPS_TOPIC['CUSTOMER_RESPONSE'].iloc[i]

    if not input_text.strip():
        print(f"Skipping empty or whitespace 'CUSTOMER_RESPONSE' at index {i}.")
        continue

    try:
        model_dict = classifier(input_text, subtopics_labels, multi_label=True)
        result_dict = dict(zip(model_dict.get('labels'), model_dict.get('scores')))
        score_dict = {'CUSTOMER_RESPONSE': input_text}  # using input_text directly
        score_dict.update(result_dict)
        detailed_scores.append(score_dict)

        print("Input Text for the model: ", input_text)
        for label in subtopics_labels:
            print(f"Confidence Score for {label}: ", result_dict.get(label, "N/A"))  # Default to "N/A" if label not found
        print('\n\n')
    except Exception as e:
        # Log the exception and skip this iteration
        print(f"An error occurred at index {i} with text: {input_text}. Error: {e}")
        continue


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Input Text for the model:  The healthcare practitioners I saw made me gave the rating
Confidence Score for nurses:  0.5879553556442261
Confidence Score for doctors:  0.9887176156044006
Confidence Score for surgeon:  0.16812407970428467
Confidence Score for front desk:  0.00040039923624135554
Confidence Score for place hygiene:  0.012658937834203243
Confidence Score for quality of building:  0.0004877753963228315
Confidence Score for cleanliness:  0.0826110765337944
Confidence Score for reception:  0.3217806816101074
Confidence Score for Unavailable drug for pick-up at pharmacy:  0.02191510610282421
Confidence Score for medication pickup related:  0.302663654088974



Input Text for the model:  The clinic attend to me promptly
Confidence Score for nurses:  0.3805404603481293
Confidence Score for doctors:  0.9586434960365295
Confidence Score for surgeon:  0.0009899449069052935
Confidence Score for front desk:  0.00028130607097409666
Confidence Score for place hygiene:  0.5117296576499939

In [ ]:
details_score_data = pd.DataFrame(detailed_scores)

In [ ]:
details_score_data.columns

Index(['CUSTOMER_RESPONSE', 'doctors', 'nurses', 'reception',
       'medication pickup related', 'surgeon', 'cleanliness',
       'Unavailable drug for pick-up at pharmacy', 'place hygiene',
       'quality of building', 'front desk'],
      dtype='object')

In [ ]:
detailed_scores_df['Medical_Team_And_Front_Desk'] = ((details_score_data['doctors']>0.9).astype(int)|
                                        (details_score_data['nurses']>0.9).astype(int)|
                                        (details_score_data['surgeon']>0.9).astype(int)|
                                        (details_score_data['front desk']>0.9).astype(int)
                                        )
detailed_scores_df['medication_related'] = ((details_score_data['Unavailable drug for pick-up at pharmacy']>0.9).astype(int)|
                                        (details_score_data['medication pickup related']>0.9).astype(int)
                                        )

detailed_scores_df['medical_facility_quality'] = ((details_score_data['reception']>0.9).astype(int)|
                                        (details_score_data['cleanliness']>0.9).astype(int)
                                        )

In [ ]:
detailed_scores_df_COI = detailed_scores_df[['medical_facility_quality',
                                             'medication_related',
                                             'Medical_Team_And_Front_Desk',
                                            'CUSTOMER_RESPONSE']]

In [ ]:
detailed_scores_df_COI

,medical_facility_quality,medication_related,Medical_Team_And_Front_Desk,CUSTOMER_RESPONSE
0,0.0,0.0,1.0,The healthcare practitioners I saw made me gave the rating
1,0.0,0.0,1.0,The clinic attend to me promptly
2,0.0,0.0,0.0,"The staff and doctors are nice. Their services is also good. \nThough on my last visit, they refused administering some test."
3,1.0,0.0,0.0,Great reception and greater service generally
4,0.0,0.0,1.0,The doctor wasn't around when I came.. But came after 5 minutes
5,0.0,0.0,0.0,Services was top notch
6,0.0,0.0,0.0,I love their services.
7,0.0,0.0,0.0,"The response rate was swift, they staff friendly, courteous, and professional."
8,0.0,0.0,1.0,The doctor was very friendly and quiet resourceful
9,0.0,0.0,0.0,Lovely serenity\nGreat medical team\nThey all seem to know what they are doing unlike many hospitals I have been


In [ ]:
NPS_TOPIC.head(3)

,CUSTOMER_RESPONSE,PROVIDER_NAME,Health_benefits_coverage,provider_quality,RCC_quality,provider_wait_times,customer_education,provider_network,medication_delivery,TYPE_SERVICE,GYM_Service,DIAGNOSITC_Service,PHARMACY_Service,OPTICAL_Service,DENTAL_Service,DERMATOLOGY_Service,ORTHOPEDIC_Service,HOSPITAL_Service,CLINIC_Service
0,The healthcare practitioners I saw made me gave the rating,Grandville Medville and Laser,0,1,0,0,0,0,0,Hospital,0,0,0,0,0,0,0,1,0
2,The clinic attend to me promptly,PEAK HEALTH HOSPITAL,0,1,0,1,0,0,0,Hospital,0,0,0,0,0,0,0,1,0
7,"The staff and doctors are nice. Their services is also good. \nThough on my last visit, they refused administering some test.",Cri Medi Clinic- VGC,0,1,0,0,0,0,0,Clinic,0,0,0,0,0,0,0,0,1


In [ ]:
NPS_TOPIC_SubTopcis = pd.merge(detailed_scores_df_COI, NPS_TOPIC, left_on = 'CUSTOMER_RESPONSE',
                          right_on = 'CUSTOMER_RESPONSE')

In [ ]:
NPS_TOPIC_SubTopcis.to_csv('NPS_TOPICS_AND_SUBTOPICS.csv')

In [ ]:
NPS_TOPIC_SubTopcis.columns

Index(['medical_facility_quality', 'medication_related',
       'Medical_Team_And_Front_Desk', 'CUSTOMER_RESPONSE', 'PROVIDER_NAME',
       'Health_benefits_coverage', 'provider_quality', 'RCC_quality',
       'provider_wait_times', 'customer_education', 'provider_network',
       'medication_delivery', 'TYPE_SERVICE', 'GYM_Service',
       'DIAGNOSITC_Service', 'PHARMACY_Service', 'OPTICAL_Service',
       'DENTAL_Service', 'DERMATOLOGY_Service', 'ORTHOPEDIC_Service',
       'HOSPITAL_Service', 'CLINIC_Service'],
      dtype='object')

In [ ]:
columns_to_check = ['provider_quality', 'RCC_quality','provider_wait_times', 'customer_education', 'provider_network',
       'medication_delivery']
def get_label(row):
    labels = [col for col in columns_to_check if row[col] == 1]
    return labels if labels else ["general"]

NPS_TOPIC_SubTopcis['Topics_lables'] = NPS_TOPIC_SubTopcis.apply(get_label, axis=1)

In [ ]:
NPS_TOPIC_SubTopcis.dropna(inplace=True)

In [ ]:
NPS_TOPIC_SubTopcis.loc[NPS_TOPIC_SubTopcis['CUSTOMER_RESPONSE'].str.contains("doctors", case=False, na=False), 'Medical_Team_And_Front_Desk'] = 1

In [ ]:
columns_to_check = ['CLINIC_Service', 'HOSPITAL_Service','ORTHOPEDIC_Service', 'DERMATOLOGY_Service', 'DENTAL_Service',
       'OPTICAL_Service','PHARMACY_Service', 'DIAGNOSITC_Service', 'GYM_Service']
def get_label(row):
    labels = [col for col in columns_to_check if row[col] == 1]
    return labels if labels else ["general"]

NPS_TOPIC_SubTopcis['Provider_Service'] = NPS_TOPIC_SubTopcis.apply(get_label, axis=1)

In [ ]:
columns_to_check = ['medical_facility_quality', 'medication_related',
       'Medical_Team_And_Front_Desk']
def get_label(row):
    labels = [col for col in columns_to_check if row[col] == 1]
    return labels if labels else ["general"]

NPS_TOPIC_SubTopcis['Proivder_Quality_Subtopics'] = NPS_TOPIC_SubTopcis.apply(get_label, axis=1)

In [ ]:
NPS_TOPIC_SubTopcis.to_csv('Pre-Check_NPS_TOPICS_SUBTOPICS.csv')

In [ ]:
NPS_TOPIC_SubTopcis['Topics_lables'] = NPS_TOPIC_SubTopcis['Topics_lables'].apply(lambda x: ', '.join(x) if x else "can't find")
NPS_TOPIC_SubTopcis['Provider_Service'] = NPS_TOPIC_SubTopcis['Provider_Service'].apply(lambda x: ', '.join(x) if x else "can't find")
NPS_TOPIC_SubTopcis['Proivder_Quality_Subtopics'] = NPS_TOPIC_SubTopcis['Proivder_Quality_Subtopics'].apply(lambda x: ', '.join(x) if x else "can't find")


In [ ]:
NPS_TOPIC_SubTopcis['Subtopics_labels'] = NPS_TOPIC_SubTopcis['Provider_Service'] + ' & ' + NPS_TOPIC_SubTopcis['Proivder_Quality_Subtopics']

In [ ]:
NPS_TOPIC_SubTopcis[['CUSTOMER_RESPONSE','Topics_lables','Subtopics_labels']].to_csv('NPS_TOPIC_SubTopics.csv')

## Handling NPS Data with rule based

In [ ]:
import pandas as pd

In [ ]:
NPS_TOPICS = pd.read_csv('/content/NPS_TOPIC_SubTopics-2.csv', index_col = 0)

In [ ]:
Qualitive_NPS_TOPICS = NPS_TOPICS.head(50)

In [ ]:
Qualitive_NPS_TOPICS['Subtopics_labels'] = Qualitive_NPS_TOPICS['Subtopics_labels'].apply(lambda x: 'Dental Service' if 'CLINIC_Service, DENTAL_Service' in x else x)
Qualitive_NPS_TOPICS['Subtopics_labels'] = Qualitive_NPS_TOPICS['Subtopics_labels'].str.replace(' & general', '', regex=False)

<ipython-input-17-da495b83d9a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Qualitive_NPS_TOPICS['Subtopics_labels'] = Qualitive_NPS_TOPICS['Subtopics_labels'].apply(lambda x: 'Dental Service' if 'CLINIC_Service, DENTAL_Service' in x else x)
<ipython-input-17-da495b83d9a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Qualitive_NPS_TOPICS['Subtopics_labels'] = Qualitive_NPS_TOPICS['Subtopics_labels'].str.replace(' & general', '', regex=False)


In [ ]:
NPS_TOPICS

,CUSTOMER_RESPONSE,Topics_lables,Subtopics_labels
0,The healthcare practitioners I saw made me gave the rating,provider_quality,HOSPITAL_Service & Medical_Team_And_Front_Desk
1,The clinic attend to me promptly,"provider_quality, provider_wait_times",HOSPITAL_Service & Medical_Team_And_Front_Desk
2,"The staff and doctors are nice. Their services is also good. \nThough on my last visit, they refused administering some test.",provider_quality,CLINIC_Service & Medical_Team_And_Front_Desk
3,Great reception and greater service generally,provider_quality,OPTICAL_Service & medical_facility_quality
4,The doctor wasn't around when I came.. But came after 5 minutes,provider_quality,HOSPITAL_Service & Medical_Team_And_Front_Desk
5,Services was top notch,general,HOSPITAL_Service & general
6,I love their services.,provider_quality,OPTICAL_Service & general
7,"The response rate was swift, they staff friendly, courteous, and professional.",provider_quality,HOSPITAL_Service & general
8,The doctor was very friendly and quiet resourceful,provider_quality,HOSPITAL_Service & Medical_Team_And_Front_Desk
9,Yes they very very accommodating and very respectful in the case of treatment they're serious about what they do there staff are calming always so i love visiting them always,provider_quality,HOSPITAL_Service & Medical_Team_And_Front_Desk


In [ ]:
import pandas as pd

In [ ]:
Quality_NPS = pd.read_csv('/content/Quality_NPS.csv')

In [ ]:
Quality_NPS = Quality_NPS.head(50)

In [ ]:
Quality_NPS.to_csv('Quality_NPS.csv')